# Functionality: visualization of functional connectivity

<ul>
    <li>visualize the ciCOH in mild and normal state</li>
</ul>

## Import Packages 

In [1]:
import os, sys
import numpy as np
import pandas as pd

import pickle

from igraph.drawing.text import TextDrawer
import cairo

In [34]:
# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)


# import animal_folder function in util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual


from simulated import ciCOH_threshold

## Preparison

### exp subfolders & code_corresfolder

In [3]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [4]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder,correparentfolder = code_corresfolder(os.getcwd(), nb_name)

### global parameters

In [5]:
# frequency
freq = [26, 28]
strfreq = 'freq' + str(freq[0]) + '_' + str(freq[1])


# eventphase
event = 'rest'


sameDates4SKTRest = True

In [6]:
### text setup for brain areas ###
texts_area = dict()

texts_area['VPLo'] = [20, 40, 20]

texts_area['VLo'] = [20, 120, 20]

texts_area['VA'] = [20, 200, 20]

texts_area['SMA'] = [20, 320, 20]

texts_area['M1'] = [20, 500, 20]

texts_area['STN'] = [280, 140, 20]

texts_area['GP'] = [360, 250, 20]

In [7]:
areas_unwanted = ['lCd','rMC']

In [8]:
WithinM1NoShow = False
STNConnectOnly = False
GPConnectOnly = False
CorticalConnectOnly = False
halfs = False

### input setup

In [9]:
inputfolder = os.path.join(correparentfolder, 'm1_calcciCOH_rest')
filename =  'ciCOH_rest' + '_freq' + str(freq[0]) + '_' + str(freq[1])

if halfs:
    filename = filename + '_halfs'
    
if sameDates4SKTRest:
    filename = filename + '_samedays'

file = os.path.join(inputfolder, filename + '.pickle')

### save setup

In [10]:
savefolder = corresfolder
savefile_prefix_origin = 'ciCOH_' + event + '_' + strfreq

if halfs:
    savefile_prefix_origin = savefile_prefix_origin + 'halfs'

    
    
if sameDates4SKTRest:
    filename = savefile_prefix_origin + '_samedays'

## Load ciCOH and Channel

In [11]:
fp = open(file, 'rb')
ciCOH = pickle.load(fp)

In [12]:
# ciCOH
ciCOH_normal, ciCOH_mild  = ciCOH['normal'], ciCOH['mild'], 
#ciCOH_moderate = ciCOH['moderate']

# channel information
chnInf_recchni, chnInf_matchni = ciCOH['chnInf_recchni'], ciCOH['chnInf_matchni']
chnInf_brainarea, chnInf_coord = ciCOH['chnInf_brainarea'], ciCOH['chnInf_coord']

## Del unwanted channels

In [13]:
def unwanted_chandata_del(ciCOH, areas_unwanted, chnInf_recchni, chnInf_brainarea, chnInf_coord):
    
    chns_unwanted = []
    for area_unwanted in areas_unwanted:
        for i, area in enumerate(chnInf_brainarea):
            if area == area_unwanted:
                chns_unwanted.append(i)
    
    nchns = ciCOH.shape[0]
    ciCOH = np.delete(ciCOH, chns_unwanted, axis = 0)
    ciCOH = np.delete(ciCOH, chns_unwanted, axis = 1)
    
    
    chnInf_recchni = [i for j, i in enumerate(chnInf_recchni) if j not in chns_unwanted]
    chnInf_brainarea = [i for j, i in enumerate(chnInf_brainarea) if j not in chns_unwanted]
    chnInf_coord = [i for j, i in enumerate(chnInf_coord) if j not in chns_unwanted]

    return ciCOH, chnInf_recchni, chnInf_brainarea, chnInf_coord

In [14]:
ciCOH_normal, chnInf_recchni, chnInf_brainarea, chnInf_coord = \
        unwanted_chandata_del(ciCOH_normal, areas_unwanted, chnInf_recchni, chnInf_brainarea, chnInf_coord)

ciCOH_mild, chnInf_recchni, chnInf_brainarea, chnInf_coord = \
        unwanted_chandata_del(ciCOH_mild, areas_unwanted, chnInf_recchni, chnInf_brainarea, chnInf_coord)

#ciCOH_moderate, chnInf_recchni, chnInf_brainarea, chnInf_coord = \
#        unwanted_chandata_del(ciCOH_moderate, areas_unwanted, chnInf_recchni, chnInf_brainarea, chnInf_coord)

In [15]:
ciCOH_normal_origin, ciCOH_mild_origin  = ciCOH_normal, ciCOH_mild
#ciCOH_moderate_origin = ciCOH_moderate

## Show Selection 

### Only Show Cortical Areas

In [16]:
if CorticalConnectOnly:
    ciCOH_normal, ciCOH_mild, ciCOH_moderate = ciCOH_normal_origin, ciCOH_mild_origin, ciCOH_moderate_origin
    
    indices = [idx for idx, area in enumerate(chnInf_brainarea) if area != 'M1' and 'SMA' not in area]
    
    # only keep the ciCOH values within M1 and SMA
    ciCOH_normal[indices, :] = 0
    ciCOH_normal[:, indices] = 0
    
    ciCOH_mild[indices, :] = 0
    ciCOH_mild[:, indices] = 0
    
    ciCOH_moderate[indices, :] = 0
    ciCOH_moderate[:, indices] = 0
    
    
    ciCOH_normal_onlyCortical, ciCOH_mild_onlyCortical, ciCOH_moderate_onlyCortical = ciCOH_normal, ciCOH_mild, ciCOH_moderate
    
    del ciCOH_normal, ciCOH_mild, ciCOH_moderate

### Not Show Connections inside M1

In [17]:
if WithinM1NoShow:
    
    ciCOH_normal, ciCOH_mild, ciCOH_moderate = ciCOH_normal_origin, ciCOH_mild_origin, ciCOH_moderate_origin
    
    indices = [idx for idx, area in enumerate(chnInf_brainarea) if area == 'M1']
    
    for chni in indices:
        for chnj in indices:
            ciCOH_normal[chni,chnj] = 0
            ciCOH_mild[chni,chnj] = 0
            ciCOH_moderate[chni,chnj] = 0
    
    savefile_prefix = savefile_prefix + '_noWithinM1'
    ciCOH_normal_noWithinM1, ciCOH_mild_noWithinM1, ciCOH_moderate_noWithinM1 = ciCOH_normal, ciCOH_mild, ciCOH_moderate
    
    del ciCOH_normal, ciCOH_mild, ciCOH_moderate

### Only show DBS Connections

In [18]:
if STNConnectOnly:
    
    indices = [idx for idx, area in enumerate(chnInf_brainarea) if area == 'STN']
    
    # only keep the ciCOH values for STN
    ciCOH_normal, ciCOH_mild = np.zeros(ciCOH_normal_origin.shape), np.zeros(ciCOH_mild_origin.shape)
    ciCOH_moderate = np.zeros(ciCOH_moderate_origin.shape)
    
    ciCOH_normal[indices, :] = ciCOH_normal_origin[indices,:]
    ciCOH_normal[:, indices] = ciCOH_normal_origin[:,indices]
    
    ciCOH_mild[indices, :] = ciCOH_mild_origin[indices,:] 
    ciCOH_mild[:, indices] = ciCOH_mild_origin[:,indices]
    
    ciCOH_moderate[indices, :] = ciCOH_moderate_origin[indices,:] 
    ciCOH_moderate[:, indices] = ciCOH_moderate_origin[:,indices]
    
    
    ciCOH_normal_onlySTN, ciCOH_mild_onlySTN, ciCOH_moderate_onlySTN = ciCOH_normal, ciCOH_mild, ciCOH_moderate
    
    del ciCOH_normal, ciCOH_mild, ciCOH_moderate

In [19]:
if GPConnectOnly:
    
    indices = [idx for idx, area in enumerate(chnInf_brainarea) if area == 'GP']
    
    # only keep the ciCOH values for GP
    ciCOH_normal, ciCOH_mild = np.zeros(ciCOH_normal_origin.shape), np.zeros(ciCOH_mild_origin.shape)
    ciCOH_moderate = np.zeros(ciCOH_moderate_origin.shape)
    
    ciCOH_normal[indices, :] = ciCOH_normal_origin[indices,:]
    ciCOH_normal[:, indices] = ciCOH_normal_origin[:,indices]
    
    ciCOH_mild[indices, :] = ciCOH_mild_origin[indices,:] 
    ciCOH_mild[:, indices] = ciCOH_mild_origin[:,indices]
    
    ciCOH_moderate[indices, :] = ciCOH_moderate_origin[indices,:] 
    ciCOH_moderate[:, indices] = ciCOH_moderate_origin[:,indices]
    
    
    ciCOH_normal_onlyGP, ciCOH_mild_onlyGP, ciCOH_moderate_onlyGP = ciCOH_normal, ciCOH_mild, ciCOH_moderate
    
    del ciCOH_normal, ciCOH_mild, ciCOH_moderate

## ciCOH Visual

In [26]:
def ciCOH_visual_save(ciCOH, chn_area, chn_coord, str_recchnishow, lowweight, tobesavedfile, texts, threds = None):
    """
        
        Args:
            
            ciCOH (np.sdarray): ciCOH matrix (nchns, nchns)
            
            chn_area (list): a list of string representing brain area for each channel, len(chn_area) = nchns
            
            chn_coord (numpy.ndarray): x, y coordinates for each channel, shape = (nchns, 2)
            
            chn_recchni(list): showed recording chni for each channel, shape = (nchns, )
            
            lowweight: the threshold lowweight, only weight>lowweight is treated as connection
            
            tobesavedfile: file to save the visualized figure
            
            texts:
            
        Output:
            the visualizaton of ciCOH is saved in tobesavedfile
            
    """
    
    weight = abs(ciCOH)

    # weight > lowweight
    weight[np.where(weight<lowweight)] =0
    
    
    # create new graph
    graph = graphVisual.graph_create(weight, vsarea = chn_area, vscoord = chn_coord, vschni = str_recchnishow)

    
    # set graph visualization style
    visual_style = dict()
    if threds is not None:
        colors_opt = ['gray', '#ff7f0e','blue' ,'#d62728']
        
        colors = ['black'] * len(graph.es['weight'])
        for i, w in enumerate(graph.es['weight']):
            if w > threds[2]: 
                colors[i] = colors_opt[3]
            elif w > threds[1]:
                colors[i] = colors_opt[2]
            elif w > threds[0]:
                colors[i] = colors_opt[1]
                
        visual_style['edge_color'] = colors

    visual_style = graphVisual.graph_style(graph, visual_style)
    

     # plot graph
    igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
    
    # save graph
    igplot.save(tobesavedfile)
    
    return igplot

## Save

In [21]:
# threshold lowweight
lowweight = 0.15

In [22]:
savefile_prefix = savefile_prefix_origin

if STNConnectOnly:
    ciCOH_normal, ciCOH_mild, ciCOH_moderate = ciCOH_normal_onlySTN, ciCOH_mild_onlySTN, ciCOH_moderate_onlySTN
    savefile_prefix = savefile_prefix_origin + '_onlySTN'

if GPConnectOnly:
    ciCOH_normal, ciCOH_mild, ciCOH_moderate = ciCOH_normal_onlyGP, ciCOH_mild_onlyGP, ciCOH_moderate_onlyGP
    savefile_prefix = savefile_prefix_origin + '_onlyGP'
    
if CorticalConnectOnly:
    ciCOH_normal, ciCOH_mild, ciCOH_moderate = ciCOH_normal_onlyCortical, ciCOH_mild_onlyCortical, ciCOH_moderate_onlyCortical
    savefile_prefix = savefile_prefix_origin + '_onlyCortical'

In [25]:
xmax = max(np.amax(abs(ciCOH_normal)), np.amax(abs(ciCOH_mild)))
threds = [lowweight, lowweight + (xmax - lowweight)/3, lowweight + (xmax - lowweight) * 2/3]
threds = np.around(threds, decimals=2)

7921


NameError: name 'ntrials' is not defined

In [ ]:
ciCOH_threshold.threshold_ciCOH_sin(ntimes = ntimes, ntrials = )

### normal

In [27]:
"""
    normal condition
"""


# save file config
savefilename = savefile_prefix + '_lowweight' + str(lowweight) + '_normal' + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()


# visualize and save
igplot = ciCOH_visual_save(ciCOH_normal, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = chnInf_recchni,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts,
                           threds = threds)

igplot.show()

print("Figure saved to " + savefile)

del savefilename, savefile, texts, igplot

Figure saved to /home/lingling/Insync/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/1_dataAnaly/FCAnaly/Rest/m2_visual_ciCOH_rest/ciCOH_rest_freq26_28_lowweight0.15_normal.png


### mild

In [30]:
"""
    mild condition
"""


condition = 'mild'


# save file config
savefilename = savefile_prefix + '_lowweight' + str(lowweight) + '_' + condition + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()


# visualize and save
igplot = ciCOH_visual_save(ciCOH_mild, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = chnInf_recchni,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts,
                           threds = threds)

igplot.show()

print("Figure saved to " + savefile)

Figure saved to /home/lingling/Insync/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/1_dataAnaly/FCAnaly/Rest/m2_visual_ciCOH_rest/ciCOH_rest_freq26_28_lowweight0.15_mild.png


### moderate

In [ ]:
"""
    moderate condition
"""

Moderate = False

if Moderate:

    condition = 'moderate'
    ciCOH = ciCOH_moderate

    # save file config
    savefilename = savefile_prefix + '_lowweight' + str(lowweight) + '_' + condition + '.png'
    savefile = os.path.join(savefolder, savefilename)

    # printed text config
    texts = texts_area.copy()


    # visualize and save
    igplot = ciCOH_visual_save(ciCOH, 
                               chn_area = chnInf_brainarea, 
                               chn_coord = chnInf_coord,
                               str_recchnishow = chnInf_recchni,
                               lowweight = lowweight,  
                               tobesavedfile = savefile, 
                               texts = texts)

    igplot.show()

    print("Figure saved to " + savefile)

    del condition, ciCOH, savefilename, savefile, texts